# การติดตั้ง scikit-surprise — คำแนะนำด่วน

แนะนำให้ใช้ conda (conda-forge) เพราะเป็นวิธีที่เสถียรบน Windows; หากไม่ได้ใช้ conda ให้ใช้ pipwin เพื่อดึง prebuilt wheel.

1) ถ้ามี Conda (แนะนำ): สร้าง environment ด้วย Python 3.11 แล้วติดตั้งจาก conda-forge (PowerShell):

```powershell
conda create -n surprise-env python=3.11 -y
conda activate surprise-env
conda install -c conda-forge scikit-surprise -y
```

หลังติดตั้ง ให้เปิด Jupyter/VSCode kernel แบบเดียวกับ environment นี้ (หรือติดตั้ง ipykernel ลงใน env และเลือก kernel) แล้วรันเซลล์ในโน้ตบุ๊กอีกครั้ง.

2) ถ้าไม่มี conda (pip / Windows): ใช้ pipwin เพื่อดึง wheel (PowerShell):

```powershell
python -m pip install --upgrade pip setuptools wheel
python -m pip install pipwin
python -m pipwin install scikit-surprise
```

หากคำสั่ง `pipwin` หา wheel ไม่เจอ หรือคุณใช้ Python 3.12+ อาจไม่มี wheel พร้อมใช้งาน — ให้พิจารณาใช้ Python 3.11 หรือใช้ conda ตามข้อ 1.

3) ขั้นตอนถ้าต้อง build จาก source (แพ้ทุกทาง): ติดตั้ง Microsoft Visual C++ Build Tools แล้วรัน:

```powershell
python -m pip install scikit-surprise
```

หมายเหตุสั้นๆ:
- ถ้าติดตั้งสำเร็จ ให้ restart kernel แล้วรันเซลล์ที่พิมพ์ `surprise.__version__` เพื่อตรวจ
- ถ้าติดตั้งยังล้มเหลว ให้คัดลอกข้อความ error แล้วส่งมาให้ผม (ผมช่วยวิเคราะห์ต่อได้)


In [7]:
# Robust installer for scikit-surprise
# - Detects Python version and prints guidance if unsupported
# - Upgrades packaging tools, installs compatible numpy/Cython
# - On Windows: try `python -m pipwin install scikit-surprise` (avoids attribute issues),
#   then fall back to conda (if available) or pip --only-binary
# - On non-Windows: try pip, then conda
import sys
import subprocess
import shutil

def run(cmd, raise_on_error=True):
    print("Running:", " ".join(cmd))
    proc = subprocess.run(cmd, capture_output=True, text=True)
    if proc.stdout:
        print(proc.stdout)
    if proc.returncode != 0:
        print("ERROR:", proc.stderr)
        if raise_on_error:
            raise subprocess.CalledProcessError(proc.returncode, cmd)
    return proc

py_ver = sys.version_info
print(f"Python version: {py_ver.major}.{py_ver.minor}.{py_ver.micro}")
if (py_ver.major, py_ver.minor) >= (3, 12):
    print("Note: scikit-surprise may not have prebuilt wheels for Python 3.12+; consider using Python 3.11 or conda if installation fails.")

try:
    import surprise
    print("surprise already installed; version:", surprise.__version__)
except Exception:
    try:
        # Upgrade packaging tools
        run([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel"]) 
    except subprocess.CalledProcessError:
        print("Warning: failed to upgrade pip/setuptools/wheel; continuing")

    try:
        # Install compatible numpy and Cython first
        run([sys.executable, "-m", "pip", "install", "numpy<2", "Cython<3"]) 
    except subprocess.CalledProcessError:
        print("Warning: could not install/upgrade numpy or Cython; continuing")

    installed = False

    if sys.platform.startswith("win"):
        # Try pipwin via subprocess to avoid attribute errors from import
        try:
            print("Attempting to install via pipwin (prebuilt wheels for Windows)...")
            # ensure pipwin is installed
            run([sys.executable, "-m", "pip", "install", "pipwin"]) 
            # call pipwin as a module
            run([sys.executable, "-m", "pipwin", "install", "scikit-surprise"]) 
            installed = True
        except subprocess.CalledProcessError as e:
            print("pipwin attempt failed:", e)

        if not installed:
            # Try conda if available
            if shutil.which("conda"):
                try:
                    print("Attempting to install via conda (conda-forge)...")
                    run(["conda", "install", "-c", "conda-forge", "scikit-surprise", "-y"]) 
                    installed = True
                except subprocess.CalledProcessError as e:
                    print("conda install failed:", e)

        if not installed:
            # Try pip with binary-only (may fail if no wheel for this Python)
            try:
                print("Attempting pip install with binary-only (may fail if wheel not available)...")
                run([sys.executable, "-m", "pip", "install", "--only-binary=:all:", "scikit-surprise"]) 
                installed = True
            except subprocess.CalledProcessError as e:
                print("pip binary install failed:", e)

    else:
        # Non-Windows: try pip first
        try:
            print("Attempting pip install scikit-surprise...")
            run([sys.executable, "-m", "pip", "install", "scikit-surprise"]) 
            installed = True
        except subprocess.CalledProcessError as e:
            print("pip install failed:", e)

        if not installed and shutil.which("conda"):
            try:
                print("Attempting conda install (conda-forge)...")
                run(["conda", "install", "-c", "conda-forge", "scikit-surprise", "-y"]) 
                installed = True
            except subprocess.CalledProcessError as e:
                print("conda install failed:", e)

    if not installed:
        print("\nAll automated install attempts failed.")
        print("Recommendations:")
        print(" - Use conda: `conda install -c conda-forge scikit-surprise` in a conda environment (most reliable).")
        print(" - Or use Python 3.11 (create a virtualenv with Python 3.11) because many wheels are built for <=3.11.")
        print(" - If you must build from source, install Microsoft Visual C++ Build Tools (on Windows) and try `pip install scikit-surprise` again.")
        raise RuntimeError("scikit-surprise installation failed; see messages above for details")

    # Final import
    import importlib
    surprise = importlib.import_module("surprise")
    print("surprise version:", surprise.__version__)


Python version: 3.13.3
Note: scikit-surprise may not have prebuilt wheels for Python 3.12+; consider using Python 3.11 or conda if installation fails.
Running: c:\Users\nut98\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip setuptools wheel

Running: c:\Users\nut98\AppData\Local\Programs\Python\Python313\python.exe -m pip install numpy<2 Cython<3

Running: c:\Users\nut98\AppData\Local\Programs\Python\Python313\python.exe -m pip install numpy<2 Cython<3
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'

ERROR:   error: subprocess-ex

RuntimeError: scikit-surprise installation failed; see messages above for details